We start by importing SimPy as simulator

In [1]:
import simpy

We define our order list by

In [2]:
#order list [ [sawing time, milling time]]
orders =     [ [4,3], 
               [8,4],
               [3,7],
               [5,6],
               [6,2],
               [3,7],
               [7,2],
               [5,1],
             ]

Please note that sawing always has to come before milling for each order. Some variables to measure the performance:

In [3]:
millDiff = 0 #collect idle times of the mill
millStart = 0 #collect start time of milling
millEnd = 0 #collect end time of milling 

We calculate a simple ordered sequence for the given list:

In [4]:
def calculateSimpleOrderForMachines(orderList):
    execOrder = list()
    for i in range(len(orderList)):
        execOrder.append(i)
    return execOrder

The orders are executed

In [5]:
def runOrders(env, orderToDo, saw, mill):
    print('t: {} order {} arrives'.format(env.now, orderToDo))
    with saw.request() as reqSaw:
        yield reqSaw
        #execute order with saw
        print('t: {}: order {} start sawing'.format(env.now, orderToDo))
        yield env.timeout(orderToDo[0])
        print('t: {}: order{} finished sawing'.format(env.now, orderToDo))
    with mill.request() as reqMill:
        yield reqMill
        print('t: {}: order {} start milling'.format(env.now, orderToDo))
        global millDiff
        global millStart
        global millEnd
        millStart = env.now
        millDiff = millDiff + (millStart - millEnd)
        yield env.timeout(orderToDo[1])
        print('t: {}: order{} finished milling'.format(env.now, orderToDo))
        millEnd = env.now

Simulation environment with capacitated resources is initialised:

In [6]:
env = simpy.Environment()
mill = simpy.Resource(env, capacity=1)
saw = simpy.Resource(env, capacity=1)

Execution sequence is calculated for the order list, here, e.g., simply ascending.

In [7]:
orderedSet = calculateSimpleOrderForMachines(orders)

Simulation is initalised with the orders:

In [8]:
for i in range(len(orders)):
    env.process(runOrders(env, orders[orderedSet[i]], saw, mill))

Run and print result:

In [9]:
env.run()

print('mill idle time: {}'.format(millDiff))

t: 0 order [4, 3] arrives
t: 0 order [8, 4] arrives
t: 0 order [3, 7] arrives
t: 0 order [5, 6] arrives
t: 0 order [6, 2] arrives
t: 0 order [3, 7] arrives
t: 0 order [7, 2] arrives
t: 0 order [5, 1] arrives
t: 0: order [4, 3] start sawing
t: 4: order[4, 3] finished sawing
t: 4: order [4, 3] start milling
t: 4: order [8, 4] start sawing
t: 7: order[4, 3] finished milling
t: 12: order[8, 4] finished sawing
t: 12: order [8, 4] start milling
t: 12: order [3, 7] start sawing
t: 15: order[3, 7] finished sawing
t: 15: order [5, 6] start sawing
t: 16: order[8, 4] finished milling
t: 16: order [3, 7] start milling
t: 20: order[5, 6] finished sawing
t: 20: order [6, 2] start sawing
t: 23: order[3, 7] finished milling
t: 23: order [5, 6] start milling
t: 26: order[6, 2] finished sawing
t: 26: order [3, 7] start sawing
t: 29: order[5, 6] finished milling
t: 29: order[3, 7] finished sawing
t: 29: order [6, 2] start milling
t: 29: order [7, 2] start sawing
t: 31: order[6, 2] finished milling
t: 31: